# Day 16 - Processor simulation

The author of Advent of Code certainly likes their [machine code](https://en.wikipedia.org/wiki/Machine_code) challenges! Not only have we seen similar problems in previous years (last year had three!; days [8](../2017/Day%2008.ipynb), [18](../2017/Day%2018.ipynb) & [23](../2017/Day%2023.ipynb)), the author previously created the [Synacor challenge](https://challenge.synacor.com/), which requires you to write your own virtual CPU for the supplied machine code, first.

But they do find novel puzzles each time! Here, we are only given the [assembly language](https://en.wikipedia.org/wiki/Assembly_language) description, and are asked to count the number of possible opcode instructions. The solution is simple; run the opcode through all possible instructions, see which ones produce the expected output.

All operands can be implemented using the Python [`operator` module](https://docs.python.org/3/library/operator.html).

In [1]:
import operator
import re
from dataclasses import dataclass
from typing import Callable, Iterable, Iterator, List, Mapping, Set, Sequence, Tuple, Type, Union

@dataclass
class Opcode:
    # An opcode function takes one or two integers as input
    f: Union[Callable[[int, int], int], Callable[[int], int]]
    operands: str # 1 or 2 characters, i and r
    opcode: str = ''

    def __set_name__(self, owner: Type['CPU'], name: str) -> None:
        self.opcode = name[3:]
        owner.opcodes[self.opcode] = self

    def __repr__(self) -> str:
        return f'<opcode {self.opcode} {self.operands!r}>'

    def __call__(self, cpu: 'CPU', *ops: int) -> None:
        # operand C receives the output
        *ops, op_c = ops
        # map the oter operands to values
        value = {'r': cpu.registers.__getitem__, 'i': operator.pos}
        ops = (value[t](op) for t, op in zip(self.operands, ops))
        # execute the operand, store the integer result in the register
        # designated by C. Int is used to convert bool results to 0 or 1
        cpu.registers[op_c] = int(self.f(*ops))

def opcode(operands: str):
    """Operator operates on 1 or 2 operands
    
    Specify the operands as 'r' or 'i' for registers and immediate values.
    """
    def decorator(f):
        return Opcode(f, operands)
    return decorator

class CPU:
    registers: List[int]  # limited to 4
    opcodes: Mapping[str, Opcode] = {}

    def __init__(self):
        self.registers = [0, 0, 0, 0]
    
    # Addition
    # addr (add register) stores into register C the result of adding register
    # A and register B.
    op_addr = opcode('rr')(operator.add)
    # addi (add immediate) stores into register C the result of adding
    # register A and value B.
    op_addi = opcode('ri')(operator.add)

    # Multiplication:
    # mulr (multiply register) stores into register C the result of
    # multiplying register A and register B.
    op_mulr = opcode('rr')(operator.mul)
    # muli (multiply immediate) stores into register C the result of
    # multiplying register A and value B.
    op_muli = opcode('ri')(operator.mul)

    # Bitwise AND:
    # banr (bitwise AND register) stores into register C the result of the
    # bitwise AND of register A and register B.
    op_banr = opcode('rr')(operator.and_)
    # bani (bitwise AND immediate) stores into register C the result of the
    # bitwise AND of register A and value B.
    op_bani = opcode('ri')(operator.and_)

    # Bitwise OR:
    # borr (bitwise OR register) stores into register C the result of the
    # bitwise OR of register A and register B.
    op_borr = opcode('rr')(operator.or_)
    # bori (bitwise OR immediate) stores into register C the result of the
    # bitwise OR of register A and value B.
    op_bori = opcode('ri')(operator.or_)

    # Assignment:
    # setr (set register) copies the contents of register A into register C.
    # (Input B is ignored.)
    op_setr = opcode('r')(operator.pos)  # pos is the identity function for int
    # seti (set immediate) stores value A into register C. (Input B is
    # ignored.)
    op_seti = opcode('i')(operator.pos)  # pos is the identity function for int

    # Greater-than testing:
    # gtir (greater-than immediate/register) sets register C to 1 if value A
    # is greater than register B. Otherwise, register C is set to 0.
    op_gtir = opcode('ir')(operator.gt)
    # gtri (greater-than register/immediate) sets register C to 1 if register
    # A is greater than value B. Otherwise, register C is set to 0.
    op_gtri = opcode('ri')(operator.gt)
    # gtrr (greater-than register/register) sets register C to 1 if register A
    # is greater than register B. Otherwise, register C is set to 0.
    op_gtrr = opcode('rr')(operator.gt)

    # Equality testing:
    # eqir (equal immediate/register) sets register C to 1 if value A is equal
    # to register B. Otherwise, register C is set to 0.
    op_eqir = opcode('ir')(operator.eq)
    # eqri (equal register/immediate) sets register C to 1 if register A is
    # equal to value B. Otherwise, register C is set to 0.
    op_eqri = opcode('ri')(operator.eq)
    # eqrr (equal register/register) sets register C to 1 if register A is
    # equal to register B. Otherwise, register C is set to 0.
    op_eqrr = opcode('rr')(operator.eq)

    def test_instruction(
        self,
        registers: List[int],
        instruction: Sequence[int],
        expected: List[int]
    ) -> Iterator[Opcode]:
        for opcode in self.opcodes.values():
            self.registers[:] = registers
            opcode(self, *instruction[1:])
            if self.registers == expected:
                yield opcode

    def execute(self, instructions: Iterable[Sequence[int]]):
        opcodes = self.opcodes
        for op, operands in instructions:
            opcodes[op](self, *operands)
        return self.registers[0]

def test_opcodes(samples: Iterable[Tuple[List[int], List[int], List[int]]]) -> int:
    cpu = CPU()
    return sum(1 for test in samples if sum(1 for _ in cpu.test_instruction(*test)) > 2)

def parse_data(data: str) -> Tuple[Sequence[Tuple[List[int], List[int], List[int]]], Sequence[List[int]]]:
    sample_lines, instruction_lines = map(str.splitlines, data.split('\n\n\n\n'))
    it = iter(sample_lines)
    samples = []
    for line in it:
        if 'Before:' in line:
            registers = [int(r) for r in re.findall('\d+', line)]
            instruction = [int(i) for i in next(it).split()]
            expected = [int(r) for r in re.findall('\d+', next(it))]
            samples.append((registers, instruction, expected))
    return samples, [[int(i) for i in l.split()] for l in instruction_lines]

In [2]:
test_samples = parse_data('''\
Before: [3, 2, 1, 1]
9 2 1 2
After:  [3, 2, 2, 1]



''')[0]
assert test_opcodes(test_samples) == 1

In [3]:
import aocd

data = aocd.get_data(day=16, year=2018)
samples, instructions = parse_data(data)

In [4]:
print('Part 1:', test_opcodes(samples))

Part 1: 605


## Part 2, recover the instruction set

Now we have to map machine code to the operator names, based on the samples. If you map the first value of each instruction to the opcode names that produce the correct output, we can see that one of those mappings has just one possible operator name. There is also a set with two names, one of which is the one named in the set with a single name. 

This gave me a hunch that we can simply eliminate mapped names from the possibities of as-yet-unresolved mappings, and by this process of elimination reconstruct the full instruction set.

Executing the program is then super simple; just execute the machine code using or new-found mapping. That part is certainly going to be straightforward, as there are no jumps (so no loops) in this instruction set.

In [5]:
def determine_opcodes(samples: Sequence[Tuple[List[int], List[int], List[int]]]) -> Mapping[int, opcode]:
    cpu = CPU()
    # we'll reduce this down per sample
    possible_opcodes: Mapping[int, Set[str]] = {i: set() for i in range(16)}
    for registers, instruction, expected in samples:
        for opcode in cpu.test_instruction(registers, instruction, expected):
            possible_opcodes[instruction[0]].add(opcode.opcode)
    
    # iteratively determine what opcode goes where
    opcode_map: Mapping[int, Opcode] = {}
    assigned: Set[str] = set()
    while len(opcode_map) < 16:
        for op, opcodes in possible_opcodes.items():
            opcodes.difference_update(assigned)
            if len(opcodes) == 1:
                opcode = opcode_map[op] = opcodes.pop()
                assigned.add(opcode)
            
    return opcode_map

def execute_instructions(samples: Sequence[Tuple[List[int], List[int], List[int]]], instructions: Sequence[List[int]]) -> int:
    opcode_map = determine_opcodes(samples)
    cpu = CPU()
    return cpu.execute((opcode_map[op], operands) for op, *operands in instructions)

In [6]:
print('Part 2:', execute_instructions(samples, instructions))

Part 2: 653
